In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
np.set_printoptions(suppress=True)

In [ ]:
class MyGaussianNB(BaseEstimator, ClassifierMixin):
    
    def fit(self, X, y):
        self.X = X
        self.y = y

        self.__mean_std_dict = {}
        
        # find each unique class and for each of these classes
        classes = np.unique(y)
        for c in classes:
            # get all indicies for this class
            indices = np.where(y==c)
            
            # for each column of this class, find the mean and std
            self.__mean_std_dict[c] = {'prior_prob':(len(indices[0])/len(X))}
            
            for i in range(len(X[0])):
                self.__mean_std_dict[c][i] = {'mean':(X[indices,i].mean()), 'std':(X[indices,i].std())}
            
        return self
        
    def predict(self, X):
        
        self.X = X
        
        predictions = np.empty([0,len(X)])
        
        def conditionalProba(val, mean, stdev):
            '''
            Function that takes a value, a mean value and a standard deviation value.
            It inputs these into a formula supplied in the assignment question and outputs the
            resulting conditional probability for each value.
            '''
            
            if stdev == 0:
                if val == mean:
                    return 1
                else:
                    return 0
                
            probability = (1/(2*np.pi*(stdev**2))**0.5) * np.exp(-((val-mean)**2)/(2*(stdev**2)))
            return probability
        
        classes = self.__mean_std_dict.keys()
        
        # go through each row of the training data
        # for each row
            # make a dictionary for probabilities for each possible class
            
        for i in range(len(X)):
            probDict = {}
            currentRow = X[i]
            
            # for each possible class
                # add a list that includes the current class's prior probability to current
                # class key in the dictionary
            for c in classes:
                probDict[c] = [self.__mean_std_dict[c]['prior_prob']]
                
                # for each column value in the current row
                    # find the mean and std for this column and class in the mean_std_dict
                    # plug the current column value, the mean for this column and the std
                    # for this column into the conditionalProba() function
                    # append the value returned from this to the current class key in the
                    # probability dictionary
                for j in range(len(currentRow)):
                    val = currentRow[j]
                    mean = self.__mean_std_dict[c][j]['mean']
                    std = self.__mean_std_dict[c][j]['std']
                    probDict[c].append(conditionalProba(val, mean, std))
                    
                # when the conditional probabilities for each column have been added to the dictionary
                # get the product of each of these values and assign it to the probability dictionary
                # for the current class
                probDict[c] = np.prod(probDict[c])
                
            # normalise the probabilities for each class
            total = sum(probDict.values())
            for k in probDict.keys():
                val = probDict[k]
                probDict[k] = val/total
                
            # get the class name with the highest normalised probability from the dictionary values
            # append the class name to the list of predictions
            index = list(probDict.values()).index(max(probDict.values()))
            predictions = np.append(predictions, list(probDict.keys())[index])
            
        # return the list of predictions
        return predictions

### Process:
- I will be creating models for 3 datasets: penguins_af.csv, diabetes.csv and glassV2.csv, using the MyGaussianNB class defined above.
- As each of the datasets are small, I will evaluate each model's performance using 5-fold cross-validation. This will give a more robust evaluation of each model's performance.
- I will compare the models created by MyGaussianNB to models created with scikit-learn's GaussianNB class across the following scores:
    - accuracy
    - confusion matrices
    - precision
    - recall
    - f1 score
    - balanced accuracy score (as each of the datasets is skewed towards one or more results)

### Read penguins data and format it

In [ ]:
penguins = pd.read_csv('penguins_af.csv', index_col=0)
penguins.reset_index(drop=True, inplace=True)
y_penguins = penguins.pop('species')
penguins = pd.get_dummies(penguins)
penguins = penguins.to_numpy()

- This leaves us with a numpy array for the penguins data and a pandas series for the classes

### Create model objects for the penguins data using MyGaussianNB and GaussianNB.

In [ ]:
myPenguin = MyGaussianNB()
gaussianPenguin = GaussianNB()

### Read diabetes data and format it

In [ ]:
diabetes = pd.read_csv('diabetes.csv', index_col=0)
diabetes.reset_index(drop=True, inplace=True)
y_diabetes = diabetes.pop('neg_pos')
diabetes = pd.get_dummies(diabetes)
diabetes = diabetes.to_numpy()

### Create model objects for the diabetes data using MyGaussianNB and GaussianNB.

In [ ]:
myDiabetes = MyGaussianNB()
gaussianDiabetes = GaussianNB()

### Read glassV2 data and format it

In [ ]:
glass = pd.read_csv('glassV2.csv', index_col=0)
glass.reset_index(drop=True, inplace=True)
y_glass = glass.pop('Type')
glass = pd.get_dummies(glass)
glass = glass.to_numpy()

### Create model objects for the glass data using MyGaussianNB and GaussianNB.

In [ ]:
myGlass = MyGaussianNB()
gaussianGlass = GaussianNB()

### For each dataset, I will use 5-fold cross validation to evaluate each model in terms of its accuracy; confusion rate; precision and recall rates; and the balanced accuracy and balanced error rates.

In [ ]:
# penguins
data = "penguins"
models = [myPenguin, gaussianPenguin]
X = penguins
y = y_penguins

print(f'Classes are {y.unique()}')

for m in models:
    print("------------------------------------")
    print(type(m).__name__, data, "data:")
    print("------------------------------------")
    accuracy = cross_val_score(m, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    print(f"accuracy: {accuracy.mean()}")
    
    y_pred = cross_val_predict(m, X, y, cv=5)
    conf_mat = confusion_matrix(y, y_pred)
    print(f"confusion matrix: \n{conf_mat}")
    
    precision = precision_score(y, y_pred, average='weighted')
    print(f"precision score: {precision}")
    
    recall = recall_score(y, y_pred, average='weighted')
    print(f"recall score: {recall}")
    
    f1 = f1_score(y, y_pred, average='weighted')
    print(f"f1 score: {f1}")
    
    bas = balanced_accuracy_score(y, y_pred)
    print(f"balanced accuracy score: {bas}")
    print("------------------------------------")
    
    print()

## penguins data conclusions:
- As we can see, the MyGaussianNB classifier outperforms scikit-learn's GaussianNB classifier in every metric.
- From the confusion matrix we can see that difference in performance mainly occurs in the prediction of the second class: Gentoo. While the GaussianNB class predicts 93 actual Adelie classes, and 52 Gentoo classes that are actually Adelie classes, the MyGaussianNB classifier predicts 142 actual Adelie classes, and just 4 Gentoo classes that are actually Adelie.

In [ ]:
# diabetes
data = "diabetes"
models = [myDiabetes, gaussianDiabetes]
X = diabetes
y = y_diabetes

print(f'Classes are {y.unique()}')

for m in models:
    print("------------------------------------")
    print(type(m).__name__, data, "data:")
    print("------------------------------------")
    accuracy = cross_val_score(m, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    print(f"accuracy: {accuracy.mean()}")
    
    y_pred = cross_val_predict(m, X, y, cv=5)
    conf_mat = confusion_matrix(y, y_pred)
    print(f"confusion matrix: \n{conf_mat}")
    
    precision = precision_score(y, y_pred, average='weighted')
    print(f"precision score: {precision}")
    
    recall = recall_score(y, y_pred, average='weighted')
    print(f"recall score: {recall}")
    
    f1 = f1_score(y, y_pred, average='weighted')
    print(f"f1 score: {f1}")
    
    bas = balanced_accuracy_score(y, y_pred)
    print(f"balanced accuracy score: {bas}")
    print("------------------------------------")
    
    print()

## diabetes data conclusions:
- As we can see, the predictions for both of the classifiers for the diabetes data is identical. This is noteworthy, as this only data has two possible classes. I will try the classifiers on another dataset with 2 possible classes to see if the results are the same in that case. 

#### Below, I took the penguins dataset with only 2 of the classes – Adelie and Gentoo – and ran the corss validation metrics on them to see the scores

In [ ]:
penguins = pd.read_csv('penguins_af.csv', index_col=0)
penguins.reset_index(drop=True, inplace=True)
y_penguins = penguins.pop('species')
penguins = pd.get_dummies(penguins)
penguins = penguins.to_numpy()
y_penguins.value_counts()

In [ ]:
y_penguins2 = pd.Series(list(y_penguins[y_penguins=='Adelie']) + list(y_penguins[y_penguins=='Gentoo']))
index = list(y_penguins[y_penguins=='Adelie'].index) + list(y_penguins[y_penguins=='Gentoo'].index)
penguins2 = penguins[index]

In [ ]:
myPenguin2 = MyGaussianNB()
gaussianPenguin2 = GaussianNB()

In [ ]:
# penguins2
data = "penguins2"
models = [myPenguin2, gaussianPenguin2]
X = penguins2
y = y_penguins2

print(f'Classes are {y.unique()}')

for m in models:
    print("------------------------------------")
    print(type(m).__name__, data, "data:")
    print("------------------------------------")
    accuracy = cross_val_score(m, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    print(f"accuracy: {accuracy.mean()}")
    
    y_pred = cross_val_predict(m, X, y, cv=5)
    conf_mat = confusion_matrix(y, y_pred)
    print(f"confusion matrix: \n{conf_mat}")
    
    precision = precision_score(y, y_pred, average='weighted')
    print(f"precision score: {precision}")
    
    recall = recall_score(y, y_pred, average='weighted')
    print(f"recall score: {recall}")
    
    f1 = f1_score(y, y_pred, average='weighted')
    print(f"f1 score: {f1}")
    
    bas = balanced_accuracy_score(y, y_pred)
    print(f"balanced accuracy score: {bas}")
    print("------------------------------------")
    
    print()

## penguins2 data conclusions:
- As we can see, both classifiers preformed very well on the penguins data liited to 2 outcomes
- The MyGaussianNB class performed slightly better – with the GaussianNB class predicting 1 Gentoo class that was actually an Adelie class

In [ ]:
# glass
data = "glass"
models = [myGlass, gaussianGlass]
X = glass
y = y_glass

print(f'Classes are {y.unique()}')

for m in models:
    print("------------------------------------")
    print(type(m).__name__, data, "data:")
    print("------------------------------------")
    accuracy = cross_val_score(m, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    print(f"accuracy: {accuracy.mean()}")
    
    y_pred = cross_val_predict(m, X, y, cv=5)
    conf_mat = confusion_matrix(y, y_pred)
    print(f"confusion matrix: \n{conf_mat}")
    
    precision = precision_score(y, y_pred, average='weighted')
    print(f"precision score: {precision}")
    
    recall = recall_score(y, y_pred, average='weighted')
    print(f"recall score: {recall}")
    
    f1 = f1_score(y, y_pred, average='weighted')
    print(f"f1 score: {f1}")
    
    bas = balanced_accuracy_score(y, y_pred)
    print(f"balanced accuracy score: {bas}")
    print("------------------------------------")
    
    print()

## Glass data conclusions:
- As we can see, the MyGaussianNB class performed better across all metrics. 
- This can be seen particularly in the predictions of the '1' class: the MyGaussianNB class predicted 47 actual classes, whereas the GaussianNB class predicted the 35 of the actual '1' class.

## Final conclusions:
- The MyGaussianNB class performed notably better than the GaussianNB class in all multiclass datasets. In binary class datasets, the results were much closer: identical in the diabetes dataset and 1 prediction in the difference (in favour of the MyGaussianNB class) for the penguins dataset with 2 outcomes.
- This leads to my over all conclusion that the MyGaussianNB classifier is a more accurate classifier than the scikit-learn GaussianNB classifier.